In [ ]:
import csv
import pandas as pd
import polars as pl
import os
from pathlib import Path
import sys
sys.path.append('/home/matebarey/Downloads/repos/mozity')
import re
from itertools import islice, chain
from src.csvengine import ParseCSV, SafeBinder
from functools import reduce , wraps
import unicodedata
import io

In [ ]:
os.chdir('/home/matebarey/Downloads/repos/mozity')


def load_csv(file_path: str):
    raw = Path(file_path).read_bytes()

    transform_map = {
        b'\r\n': b'\n',       # Windows line endings
        b'\r': b'\n',         # Mac line endings
        b'\x00': b'',         # Null bytes
        b'\xef\xbb\xbf': b''  # BOM
    }

    cleaned = reduce(lambda acc, kv: acc.replace(*kv), transform_map.items(), raw)
    text = cleaned.decode('utf-8', errors='ignore')
    lines = text.splitlines()

    skip = 1 if lines and lines[0].lower().startswith('sep=') else 0
    sample = '\n'.join(lines[skip:skip + 10])

    header_guess = SafeBinder(sample).bind(lambda s: csv.Sniffer().has_header(s))() or False
    header_opt = 0 if header_guess else None

    df = pd.read_csv(io.StringIO(text), header=header_opt, skiprows=skip, on_bad_lines='skip')
    return df, pl.from_pandas(df)



In [19]:
#load files
files = ["snippets.csv","events.csv","prompts.csv"]
cfs = [ParseCSV(i,has_headers=True) for i in files]
df_dp = [load_csv(file) for file in files]

In [ ]:
# build cli search app.py  snippets.csv --q "cafe" --fields title, body --diacritics-off

#reqs
#1) case insensitive  normalize both query and field by removing accents
#2) boolean mini dsl term , multi word , field: term , tag:resum, title:"cafe review", oparators and or unary neagtion